In [63]:
import csv

from anytree import Node, RenderTree

Commandes pour générer les fichiers `.csv` depuis le dump envoyé par la DB :
```
ls | xargs -i basename {} .xlsx | xargs -i ssconvert {}.xlsx {}.csv
find -name '*.csv' | xargs -i wc -l {}
```

In [78]:
FILE_1 = './raw_data/h2016.csv'
FILE_2 = './raw_data/h2017.csv'
EMBEDDING_FILE_PATH = '/home/alexis/mva/word_embeddings/wiki.fr.vec'

# Loading Data

In [60]:
class Line:
    def __init__(self, row):
        self.id_tms = int(row[1])
        self.tm = row[2]
        self.id_ms = row[3]
        self.ms = row[4]
        self.id_pg = int(row[5])
        self.pg = row[6]
        self.id_a = int(row[7])
        self.a = row[8]
        # Parse sous-action if exists
        self.id_sa = int(row[9]) if len(row[9]) > 0 else None
        self.sa = row[10] if len(row[10]) > 0 else None
        
        self.embedding = None
    
    def build_embedding(self, embeddings):
        pass

In [64]:
def parse_csv(file_path):
    data = []
    with open(file_path, 'r') as f:
        r = csv.reader(f, delimiter=',', quotechar='"')
        # Skip header
        next(r, None)
        # Iterate through rows
        for row in r:
            node = Line(row)
            data.append(node)
            
    return data

In [62]:
data1 = parse_csv(FILE_1)
data2 = parse_csv(FILE_2)

# Create Word Embedding

## Count maximal number of words

In [74]:
def maximal_number_words(data):
    m = 0
    for line in data:
        m = max(m, len(line.tm.split(' ')))
        m = max(m, len(line.ms.split(' ')))
        m = max(m, len(line.pg.split(' ')))
        m = max(m, len(line.a.split(' ')))
        m = max(m, len(line.sa.split(' ')) if line.sa else 0)
    return m

In [75]:
maximal_number_words(data1)

46

## Load word embeddings

In [ ]:
import gensim
from gensim.models import Word2Vec
pretrained_embeddings = gensim.models.KeyedVectors.load_word2vec_format(EMBEDDING_FILE_PATH, binary=False)